# First Look
Here we take a first look at the dataset and its features to get an overview and to develop a strategy for
the cleaning process.

In [1]:
import pandas as pd
import numpy as np
import yaml
import sys
import os

In [2]:
sys.path.insert(0, os.path.abspath('../src'))
import mylib.cleaning as clean

#### Load configs from yaml file

In [3]:
with open('../params.yaml') as file:
    config = yaml.safe_load(file)

#### Read dataset

In [31]:
data = pd.read_csv(config['data']['csv_raw'])

In [34]:
data.head(3)

,age,work_class,earning,education,education_num,marital_status,occupation,relationship,race,sex,captial_gain,capital_loss,hours_per_week,native_country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States


#### Normalize column names

In [33]:
data.columns = clean.normalize_column_names(data.columns)
print(data.columns)

Index(['age', 'work_class', 'earning', 'education', 'education_num',
       'marital_status', 'occupation', 'relationship', 'race', 'sex',
       'captial_gain', 'capital_loss', 'hours_per_week', 'native_country'],
      dtype='object')


#### Get dataset information

In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   work_class      32561 non-null  object
 2   earning         32561 non-null  int64 
 3   education       32561 non-null  object
 4   education_num   32561 non-null  int64 
 5   marital_status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  captial_gain    32561 non-null  int64 
 11  capital_loss    32561 non-null  int64 
 12  hours_per_week  32561 non-null  int64 
 13  native_country  32561 non-null  object
dtypes: int64(6), object(8)
memory usage: 3.5+ MB


Luckily we do not have any Nan values within our dataset and the datatypes seem to be correct at the first sight.

#### Take a look at the categoricals

In [21]:
data.describe(include=object).T

,count,unique,top,freq
work_class,32561,9,Private,22696
education,32561,16,HS-grad,10501
marital_status,32561,7,Married-civ-spouse,14976
occupation,32561,15,Prof-specialty,4140
relationship,32561,6,Husband,13193
race,32561,5,White,27816
sex,32561,2,Male,21790
native_country,32561,42,United-States,29170


#### Column `work_class`
- Drop rows with values 'Without-pay' or 'Never-Worked'
- Rename '?' to 'Other'

In [8]:
data['work_class'].value_counts()

 Private             22696
 Self-emp-not-inc     2541
 Local-gov            2093
 ?                    1836
 State-gov            1298
 Self-emp-inc         1116
 Federal-gov           960
 Without-pay            14
 Never-worked            7
Name: work_class, dtype: int64

#### Column `education`
- Merge '10th', '11th' and '12th'
- Merge '7th-8th' and '9th'
- Merge 'Preschool', '1st-4th' and '5th-6th'

In [9]:
data['education'].value_counts()

 HS-grad         10501
 Some-college     7291
 Bachelors        5355
 Masters          1723
 Assoc-voc        1382
 11th             1175
 Assoc-acdm       1067
 10th              933
 7th-8th           646
 Prof-school       576
 9th               514
 12th              433
 Doctorate         413
 5th-6th           333
 1st-4th           168
 Preschool          51
Name: education, dtype: int64

#### Column `marital_status`
- Merge 'Widowed', 'Married-spouse-absent' and 'Married-AF-spouse' to Other

In [10]:
data['marital_status'].value_counts()

 Married-civ-spouse       14976
 Never-married            10683
 Divorced                  4443
 Separated                 1025
 Widowed                    993
 Married-spouse-absent      418
 Married-AF-spouse           23
Name: marital_status, dtype: int64

#### Columns `occupation`
- Merge 'Priv-house-serv', 'Armed-Forces' and '?' to Other

In [11]:
data['occupation'].value_counts()

 Prof-specialty       4140
 Craft-repair         4099
 Exec-managerial      4066
 Adm-clerical         3770
 Sales                3650
 Other-service        3295
 Machine-op-inspct    2002
 ?                    1843
 Transport-moving     1597
 Handlers-cleaners    1370
 Farming-fishing       994
 Tech-support          928
 Protective-serv       649
 Priv-house-serv       149
 Armed-Forces            9
Name: occupation, dtype: int64

#### Column `relationship`
- Rename 'Other-relative' to 'Other'

In [12]:
data['relationship'].value_counts()

 Husband           13193
 Not-in-family      8305
 Own-child          5068
 Unmarried          3446
 Wife               1568
 Other-relative      981
Name: relationship, dtype: int64

#### Column `race`
- Add 'Amer-Indian-Eskimo' to 'Other'

In [13]:
data['race'].value_counts()

 White                 27816
 Black                  3124
 Asian-Pac-Islander     1039
 Amer-Indian-Eskimo      311
 Other                   271
Name: race, dtype: int64

#### Column `sex`
- Rename 'Male' to 'M'
- Rename 'Female' to 'F'

In [14]:
data['sex'].value_counts()

 Male      21790
 Female    10771
Name: sex, dtype: int64

#### Column `native_country`
- Merge 'United-States', 'Canada', 'Outlying-US(Guam-USVI-etc)' to **North-America**
- Merge 'Mexico', 'Puerto-Rico', 'El-Salvador', 'Cuba', 'Jamaica', 'Nicaragua', 'Peru', 'Guatemala', 'Honduras', 'Columbia', 'Haiti', 'Trinadad&Tobago', 'Dominican-Republic', 'Ecuador' to **South-America**
- Merge 'Philippines', 'India', 'China', 'Vietnam', 'Laos', 'Thailand', 'Taiwan', 'Cambodia', 'Japan', 'Hong', 'Iran' to **Asia**
- Merge 'Germany', 'England', 'Holand-Netherlands', 'Ireland', 'France', 'Scotland', 'Portugal', 'Italy', 'Greece', 'Yugoslavia', 'Poland', 'Hungary' to **Europe**
- Merge '?' and 'South' to **Other**
- Rename column to **continent**

In [29]:
data['native_country'].value_counts()

 United-States                 29170
 Mexico                          643
 ?                               583
 Philippines                     198
 Germany                         137
 Canada                          121
 Puerto-Rico                     114
 El-Salvador                     106
 India                           100
 Cuba                             95
 England                          90
 Jamaica                          81
 South                            80
 China                            75
 Italy                            73
 Dominican-Republic               70
 Vietnam                          67
 Guatemala                        64
 Japan                            62
 Poland                           60
 Columbia                         59
 Taiwan                           51
 Haiti                            44
 Iran                             43
 Portugal                         37
 Nicaragua                        34
 Peru                             31
 

#### Take a look at the numericals

In [40]:
data.describe()

,age,earning,education_num,captial_gain,capital_loss,hours_per_week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000
